In [39]:
import math     
import csv
import numpy as np
import time
import re
import Crypto.Util.number

In [40]:
def read_data(file_name):
    file = open(file_name, "r")
    data = list(csv.reader(file))
    file.close()
    A = np.array(data)
    return A[:,0]

In [41]:
characters=read_data('character.csv')

In [42]:
def RSA_Ecrption(plain_text,e,n):
    C=pow(int(plain_text),e,n)
    return C

In [ ]:
def mapping_character_to_num(plain_text,i):
    M=0
    for j in range(0,5,1):
        if(i+j<len(plain_text)):
              x=np.where(characters==(plain_text[i+j].lower()))
              if(len(x[0])>0):
                M+=x[0][0]*(37**(4-j))
              else:
                 M+=36*(37**(4-j))
        else:
           M+=36*(37**(4-j))
    return M

In [43]:
def Sending_Message(plain_text,e,n,socket,name):
   C=0
   for i in range(0, len(plain_text), 5):
        if(i==0):
          start = time.time()
        M=mapping_character_to_num(plain_text,i)
        C=RSA_Ecrption(M,e,n)
        time.sleep(0.01)
        socket.send(str(C).encode())
        file = open('plain_cipher_'+name+'.txt', 'a')
        file.write(str(C)+"\n")
        file.close()
   end = time.time()
   file = open('Encryption_time_of'+name+'.txt', 'a')
   file.write(str(end-start)+"\n")
   file.close()     
   return C

In [45]:
def key_generation(p,q):
  n=p*q
  phi = (p-1)*(q-1)
  e=2
  while (e < phi):
    if(math.gcd(e, phi) == 1):
        break
    else:
        e = e+1
  d =pow(e,-1,phi)
  return e,d

In [46]:
def Decryption(C,d,n):
    P =pow(C,d,n)
    return P

In [ ]:
def num_to_mapping_character(P):
     index=0
     Message=""
     for i in range(0,5):
          index=((P-index)%(37**(i+1)))//(37**i)
          Message=str(characters[int(index)])+str(Message)
     return Message    

In [47]:
def Recieve_Message(d,n,connection,name):
    end_of_message=False
    all_message=""
    while(not(end_of_message)):
        C=connection.recv(1024).decode()
        if(all_message==""):
            start = time.time()
        P=Decryption(int(C),d,n)
        Message=num_to_mapping_character(P)
        all_message=all_message+Message
        if('     'in all_message):
             end_of_message=True
             all_message=all_message[0:len(all_message)-5]
    end = time.time()
    file = open('Decryption_time_of'+name+'.txt', 'a')
    file.write(str(end-start)+"\n")
    file.close()
    return all_message

In [48]:
def checking_decryption(cipher_txt,Plain_txt,d,n):
    all_message=''
    for i in range(0,len(cipher_txt)):
        P = Decryption(cipher_txt[i],d,n)
        Message=num_to_mapping_character(P)
        all_message+=Message 
        all_message=all_message.replace(" ","")
    Plain_txt=re.sub('[^a-zA-Z0-9 \n\.]', ' ', Plain_txt.lower()).replace(" ","")
    if all_message == Plain_txt:
        print("Attacking sucessed")
        print("Plain: "+Plain_txt)
        print("Cipher: "+all_message)
    else:
        print("Attacking failed")
        print("Plain: "+Plain_txt)
        print("Cipher: "+all_message)
    return Message 

In [49]:
def getting_phi_n(n):
    phi=0
    end_of_p_range=int(math.sqrt(n))
    if(n%2==0):
            p=2
            q=n//p
            phi=(p-1)*(q-1) 
    else:
        for factor_of_n in range(3,end_of_p_range+1,2):
                if(n%factor_of_n!=0):
                    continue
                else:
                      p=factor_of_n
                      q=n//p
                      phi=(p-1)*(q-1)  
    return phi

In [50]:
def getting_d(n,e):
    phi=getting_phi_n(n)
    d =pow(e,-1,phi)
    return d

In [51]:
def n_with_different_size():
        while(1):
                bits =input("Enter Size of n more than or equal 28")
                bits=int(bits)
                if(bits>=28):
                        break
        p_bits=bits//2
        q_bits=bits-p_bits
        print ("No of bits in prime is ",bits)
        p=Crypto.Util.number.getPrime(p_bits, randfunc=Crypto.Random.get_random_bytes)
        print ("\nRandom n-bit Prime (p): ",p)
        q=Crypto.Util.number.getPrime(q_bits, randfunc=Crypto.Random.get_random_bytes)
        print ("\nRandom n-bit Prime (q): ",q)
        n=p*q
        print ("\nN=p*q=",n)
        return p,q,n
